In [1]:
import os       
import inspect
import operator
from nltk import cluster
from nltk.cluster import cosine_distance
from numpy import array
from sklearn.cluster import KMeans
import numpy as np
from sklearn.metrics.pairwise import pairwise_distances
from heapq import heappush, heappop, nlargest
import numpy
from operator import add
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd 
import matplotlib.pyplot as plt
%matplotlib inline
from copy import copy,deepcopy
from sklearn.metrics import log_loss
import scipy as sp
import scipy.stats
from math import exp
from math import log
import scipy.sparse as sps
from sklearn.preprocessing import normalize
import time
from scipy import spatial
from random import shuffle
from scipy.sparse import coo_matrix

## Read Data

In [2]:
confidential=[]
dataleak=[]
embezzlement=[]
passwords=[]
all_log_dirs = []
all_log_dirs.append("C:\\Users\\shico\\Downloads\\second\\logs\\data\\") 
confidential.append("C:\\Users\\shico\\Downloads\\second\\logs\\confidential\\")   
dataleak.append("C:\\Users\\shico\\Downloads\\second\\logs\\dataleak\\")   
embezzlement.append("C:\\Users\\shico\\Downloads\\second\\logs\\embezzlement\\")   
passwords.append("C:\\Users\\shico\\Downloads\\second\\logs\\passwords\\") #Reminder: Change the file path to your own  
#Read data files
def checkindex(local1,local2):#check if two viarables are equal
    if local1==local2:
        return 1
    else:
        return 0

def readdata(mylist,index):#Read all the data throughout the files
    confidential=1
    dataleak=2
    embezzlement=3
    passwords=4
    data=5
    temp=''
    if checkindex(confidential,index)==1:
        temp='confidential'
    if checkindex(dataleak,index)==1:
        temp='dataleak'
    if checkindex(embezzlement,index)==1:
        temp='embezzlement'
    if checkindex(passwords,index)==1:
        temp='passwords'
    if checkindex(data,index)==1:
        temp='data'
        
    wordlist=[]
    highmatrix=[]
    logDir = mylist[0]
    logFiles = next(os.walk(logDir))[2]
    #logFiles is the a list containing all the files' names
    os.chdir("C:\\Users\\shico\\Downloads\\second\\logs\\%s\\" %temp)
   
    for i in range(len(logFiles)):
        with open(logFiles[i], "r") as f:
            try:
                data = f.readlines()
            except UnicodeDecodeError:
                pass
            for line in data:
                word=line.split()
                if len(word)>3:
                    if "RCL"in word[3]:
                        #state2
                        wordlist.append(word[3]+word[4]+word[5])
                    if "Non-object" in word[3]:
                        #state1
                        wordlist.append(word[3]+word[4])
                    if "Table" in word[3]:
                        #state2
                        wordlist.append(word[3]+word[4])
                    if "mouse" in word[3]:
                        #state1
                        wordlist.append(word[3]+word[4])
                    if "Column" in word[3]:
                        #state2
                        wordlist.append(word[3]+word[4]+word[5])
                    if "Button" in word[3]:
                        #state4
                        wordlist.append(word[3]+word[4]+word[5])
                    if "Minimap" in word[3]:
                        #state5
                        if "box" in word[6]:
                            wordlist.append(word[3]+word[4]+word[5]+word[6]+word[7])
                        else:
                            wordlist.append(word[3]+word[4]+word[5]+word[6])
            highmatrix.append(wordlist)
            wordlist=[]
    return highmatrix

confidential1=readdata(confidential,1)
dataleak1=readdata(dataleak,2)
embezzlement1=readdata(embezzlement,3)
passwords1=readdata(passwords,4)
all_log_dirs1 = readdata(all_log_dirs,5)


def retrieve_name(var):
    callers_local_vars = inspect.currentframe().f_back.f_locals.items()
    return [var_name for var_name, var_val in callers_local_vars if var_val is var]



In [3]:
def transferindex(mylist)  : #This function transfers all the text to states
    #highmatrix2 is a matrix containning all the actions in each sequence
    highmatrix2=[]
    temp=[]
    for i in range(len(mylist)):
        for j in range (len(mylist[i])):
            if "Non-object" in mylist[i][j] or "mouse" in mylist[i][j] or "Buttonpush" in mylist[i][j]:
                temp.append(1)
            if "RCL" in mylist[i][j] or "Table" in mylist[i][j] or "column" in mylist[i][j]:
                temp.append(2)
            if "Buttonclick" in mylist[i][j]:
                temp.append(4)
            if "Minimap" in mylist[i][j]:
                temp.append(5)
        highmatrix2.append(temp)
        temp=[]
    return highmatrix2
confidential2=transferindex(confidential1)   
dataleak2=transferindex(dataleak1)
embezzlement2=transferindex(embezzlement1)
passwords2=transferindex(passwords1)
all_log_dirs2 =transferindex(all_log_dirs1)


In [4]:
def transferaction(mylist)  : #This function transfers all the text to actions
    #highmatrix2 is a matrix containning all the actions in each sequence
    highmatrix2=[]
    temp=[]
    for i in range(len(mylist)):
        for j in range (len(mylist[i])):
            if "Non-object" in mylist[i][j]:
                temp.append(0)
            if  "mouse" in mylist[i][j]:
                temp.append(1)
 
            if "Buttonpushed" in mylist[i][j]:
                temp.append(2)  
            if "RCLScroll" in mylist[i][j]:
                temp.append(3)

            if "Table" in mylist[i][j]:
                temp.append(4) 

            if "Columnclicked" in mylist[i][j]:
                temp.append(5) 
            if "Buttonclick:Global" in mylist[i][j]:
                temp.append(6)
            if "Buttonclick:Normal" in mylist[i][j]:
                temp.append(7)
            if "Buttonclick:Malicious" in mylist[i][j]:
                temp.append(8)
            if "Buttonclick:Suspect" in mylist[i][j]:
                temp.append(9)
            if "Minimap" in mylist[i][j]:
                if "Minimapclickonbox" not in mylist[i][j]:
                    temp.append(10)
                else:
                    temp.append(11)
        highmatrix2.append(temp)
        temp=[]
    return highmatrix2
actionconfidential=transferaction(confidential1)   
actiondataleak=transferaction(dataleak1)
actionembezzlement=transferaction(embezzlement1)
actionpasswords=transferaction(passwords1)
actionall_log_dirs =transferaction(all_log_dirs1)


In [5]:
def merge_sequence(mylist): #This function merges all the consecutive mousexxx and RCL actions
    output=[]
    temp=[]
    last=-1
    for j in mylist:
        for i in j:
            if i==last and i==1: #mousexxx  actions
                last=i
                continue
            if i==last and i==3: #RCL actions
                last=i
                continue
            last=i
            temp.append(i)
        output.append(temp)
        temp=[]
    return output
actionconfidential= merge_sequence(actionconfidential)  
actiondataleak= merge_sequence(actiondataleak) 
actionembezzlement= merge_sequence(actionembezzlement) 
actionpasswords= merge_sequence(actionpasswords)
actionall_log_dirs=merge_sequence(actionall_log_dirs)

## Kmean method & K nearest Neighbor & Weighted average

In [6]:
def create_user_transit_matrix(train_set,num_actions):#train_set's form is [[]], num_actions is the number of actions
    # This function returns a giant matrix whose colomns are Paiwise_actions and rows are different users
    
    temp=-1
    row=[]
    col=[]
    data=[]
    coartransit=[]
    for i in train_set:
        for t in i:
            if temp==-1:#Save the first data
                temp=t
            if temp!=-1:
                row.append(0)
                col.append(temp*num_actions+t)
                data.append(1)
                temp=t
        
        coartransit.append(coo_matrix((data, (row, col)), shape=(1, num_actions*num_actions)).toarray()[0])
        #This quickly returns the one user's sequence
        row=[]
        col=[]
        data=[]
  
    return coartransit
  
    
    
    
def pairwise_k_mean(new_sequence, oldset, num_actions,k,PairwiseOld):
    #new_sequence is a 1d sequence, oldset is the trainning set(form is [[],[],[],...]), num_actions is the number of actions
    #k is 'kmean's k , PairwiseOld is the matrix returned from function create_user_transit_matrix

    temp2=0
    clusterer = cluster.KMeansClusterer(k, cosine_distance)
    
   
#     vectors = [np.array(f) for f in PairwiseOld]
    clusters = clusterer.cluster(PairwiseOld, True, trace=True)
    cluster_centers=clusterer.means() # Get the cluster centers
    transfer=np.zeros(num_actions*num_actions)
    temp=-1
    for i in new_sequence: # Create the pairwise 1d vector for the new_sequence
        if temp==-1:
            temp=i
        if temp!=-1:
            transfer[temp*num_actions+i]+=1
            temp=i

    current_num=new_sequence[-1]
    
    temp=clusterer.classify(transfer) # Get the index which the new_sequence belongs to
  
    temp2=cluster_centers[temp]   # Get the cluster center vector

    testlist=temp2[((current_num)*num_actions):((current_num+1)*num_actions)] #Find the corresponding next_step

    
    
    return np.argsort(testlist) #Return a list of next state. (Lowest to Highest)
def pairwise_k_nearst_neighbor_cosine(k, new_sequence, oldset, num_actions, Pairwise):
    #new_sequence is a 1d sequence, oldset is the trainning set(form is [[],[],[],...]), num_actions is the number of actions
    #k is 'k nearest neighbor's k , PairwiseOld is the matrix returned from function create_user_transit_matrix

   

    new=np.zeros(num_actions*num_actions)
    temp=-1
    for i in new_sequence:# Create the pairwise 1d vector for the new_sequence
        if temp==-1:
            temp=i
        if temp!=-1:
            new[temp*num_actions+i]+=1
            temp=i
    temp2=1000
    distance=[]
    for i in Pairwise:
        new_item = spatial.distance.cosine(i,new)# get the distance using cosine similarity
        distance.append(new_item)
    templist=np.argsort(distance)[0:k] # get the Nearest k trajectories from the training set

    current_num=new_sequence[-1]
    
    tempstorage=np.zeros(num_actions*num_actions)

  
    for p in templist:
         tempstorage=tempstorage+np.array(Pairwise[p])
    my_new_list = [i * (1/k) for i in tempstorage]  # Add all the neighbors and average them
 
    my_new_list2=my_new_list[((current_num)*num_actions):((current_num+1)*num_actions)]
    
    return np.argsort(my_new_list2)#Return a list of next state. (Lowest to Highest)

def pairwise_weighted_average(k, new_sequence, oldset, num_actions, Pairwise):
    #new_sequence is a 1d sequence, oldset is the trainning set(form is [[],[],[],...]), num_actions is the number of actions
    #k is 'k nearest neighbor's k , PairwiseOld is the matrix returned from function create_user_transit_matrix
    new=np.zeros(num_actions*num_actions)
    temp=-1
    for i in new_sequence:# Create the pairwise 1d vector for the new_sequence
        if temp==-1:
            temp=i
        if temp!=-1:
            new[temp*num_actions+i]+=1
            temp=i
    temp2=1000
    distance=[]
    for i in Pairwise:
        new_item = spatial.distance.cosine(i,new)# get the distance using cosine similarity
        distance.append(new_item)
    templist=np.argsort(distance)[0:k] # get the Nearest k trajectories from the training set

    current_num=new_sequence[-1]
    
    tempstorage=np.zeros(num_actions*num_actions)

  
    for p in templist:
         tempstorage=tempstorage+[i * (distance[p]) for i in Pairwise[p]]
    my_new_list = [i * (1/k) for i in tempstorage] # Add all the neighbors time distances and average them
 
    my_new_list2=my_new_list[((current_num)*num_actions):((current_num+1)*num_actions)]
    
    return np.argsort(my_new_list2)

def random_shff(actionconfidential,actionembezzlement,actiondataleak,actionpasswords):
    #shffule the data_set ready for train, validation and test split
    # eight sequences for trainning set
    #eight sequences for validation set
    train1=[]
    train2=[]
    overall=[]
    for i in actionconfidential:
        overall.append(i)
    for i in actionembezzlement:
        overall.append(i)
    for i in actiondataleak:
        overall.append(i)
    for i in actionpasswords:
        overall.append(i)
    shuffle(actionconfidential)
    shuffle(actionembezzlement),
    shuffle(actiondataleak)
    shuffle(actionpasswords)
    for i in range(2):
        train1.append(actionconfidential[i])
        train1.append(actionembezzlement[i])
        train1.append(actiondataleak[i])
        train1.append(actionpasswords[i])
        overall.remove(actionconfidential[i])
        overall.remove(actionembezzlement[i])
        overall.remove(actiondataleak[i])
        overall.remove(actionpasswords[i])
    for i in range(2,4):
        train2.append(actionconfidential[i])
        train2.append(actionembezzlement[i])
        train2.append(actiondataleak[i])
        train2.append(actionpasswords[i])
        overall.remove(actionconfidential[i])
        overall.remove(actionembezzlement[i])
        overall.remove(actiondataleak[i])
        overall.remove(actionpasswords[i])
    return train1, train2, overall

## Accuracy test for the three methods

In [7]:
def kmean_accuracy(train,vali,test,num_actions,cut_number=50,k=4):
    # train,vali and test are trainning set, validation set and test set. Cut_number is the starting point for testing sequence. 
    # if starting from the beginning, change cut_number to 0
    # K=4 because for UI data there are four tasks
    success_num=0
    total=0
    for i in vali:
        test.append(i)
    Pairwise=create_user_transit_matrix(test,num_actions)
    acuracy=[]
    returnk=[]
    highest=0
    index=-1
    
    
    
    for i in train:
        length=len(i)
        for j in range(cut_number,length):
            total+=1
            if pairwise_k_mean(i[:j], test, num_actions,k,Pairwise)[-1]==i[j] :
                success_num+=1
    return success_num/total
def k_nearest_neightbor_accuracy(train,vali,test,num_actions,cut_number=50):
    # train,vali and test are trainning set, validation set and test set. Cut_number is the starting point for testing sequence. 
    # if starting from the beginning, change cut_number to 0

    success_num=0
    total=0
    Pairwise=create_user_transit_matrix(test,num_actions)
    acuracy=[]
    returnk=[]
    highest=0
    index=-1
    for k in range(1,11): # to find best k from validation set
        for i in vali:
            length=len(i)
            for j in range(cut_number,length):
                total+=1
                if pairwise_k_nearst_neighbor_cosine(k,i[:j], test, num_actions,Pairwise)[-1]==i[j] :
                    
                    success_num+=1
        print('Validating: The accuracy when K=',k,'is', success_num/total)
        if success_num/total>highest: # Get highest accuracy and its corresponding k
            highest=success_num/total
            index=k
            success_num=0
            total=0
    print("Validating: The highest accuracy is",highest,'The corresponding k is',index)
    for i in vali:
        test.append(i)
    for i in train:
        length=len(i)
        for j in range(cut_number,length):
            total+=1
            if pairwise_k_nearst_neighbor_cosine(k,i[:j], test, num_actions,Pairwise)[-1]==i[j] :#Use k to test in trainning set
                success_num+=1
    return success_num/total
def Weighted_average_accuracy(train,vali,test,num_actions,cut_number=50):
    # train,vali and test are trainning set, validation set and test set. Cut_number is the starting point for testing sequence. 
    # if starting from the beginning, change cut_number to 0
    success_num=0
    total=0
    Pairwise=create_user_transit_matrix(test,num_actions)
    acuracy=[]
    returnk=[]
    highest=0
    index=-1
    for k in range(1,11):# to find best k from validation set
        for i in vali:
            length=len(i)
            for j in range(cut_number,length):
                total+=1
                if pairwise_weighted_average(k,i[:j], test, num_actions,Pairwise)[-1]==i[j] :
                    
                    success_num+=1
        print('Validating: The accuracy when K=',k,'is', success_num/total)
        if success_num/total>highest:
            highest=success_num/total
            index=k
            success_num=0
            total=0
    print("Validating: The highest accuracy is",highest,'The corresponding k is',index)
    for i in vali:
        test.append(i)
    for i in train:
        length=len(i)
        for j in range(cut_number,length):
            total+=1
            if pairwise_weighted_average(k,i[:j], test, num_actions,Pairwise)[-1]==i[j] :#Use k to test in trainning set
                success_num+=1
    return success_num/total



In [8]:
abc=random_shff(actionconfidential,actionembezzlement,actiondataleak,actionpasswords)
x=kmean_accuracy(abc[0],abc[1],abc[2],12,cut_number=50)


k-means trial 0
iteration
iteration
iteration
iteration
k-means trial 0
iteration
iteration
iteration
iteration
iteration
iteration
iteration
iteration
k-means trial 0
iteration
iteration
iteration
iteration
iteration
iteration
iteration
k-means trial 0
iteration
iteration
iteration
iteration
iteration
iteration
iteration
iteration
iteration
iteration
k-means trial 0
iteration
iteration
iteration
iteration
iteration
iteration
iteration
k-means trial 0
iteration
iteration
iteration
iteration
iteration
iteration
k-means trial 0
iteration
iteration
iteration
iteration
iteration
k-means trial 0
iteration
iteration
iteration
iteration
iteration
iteration
iteration
iteration
iteration
k-means trial 0
iteration
iteration
iteration
iteration
iteration
iteration
iteration
k-means trial 0
iteration
iteration
iteration
iteration
iteration
iteration
iteration
k-means trial 0
iteration
iteration
iteration
iteration
iteration
iteration
iteration
k-means trial 0
iteration
iteration
iteration
iteratio

iteration
iteration
iteration
iteration
iteration
iteration
iteration
iteration
iteration
k-means trial 0
iteration
iteration
iteration
iteration
iteration
iteration
iteration
iteration
k-means trial 0
iteration
iteration
iteration
iteration
k-means trial 0
iteration
iteration
iteration
iteration
iteration
iteration
iteration
k-means trial 0
iteration
iteration
iteration
iteration
iteration
iteration
k-means trial 0
iteration
iteration
iteration
iteration
iteration
iteration
iteration
iteration
iteration
iteration
k-means trial 0
iteration
iteration
iteration
iteration
iteration
iteration
iteration
k-means trial 0
iteration
iteration
iteration
iteration
iteration
k-means trial 0
iteration
iteration
iteration
iteration
iteration
iteration
k-means trial 0
iteration
iteration
iteration
iteration
iteration
iteration
iteration
iteration
iteration
iteration
iteration
iteration
k-means trial 0
iteration
iteration
iteration
iteration
iteration
iteration
iteration
iteration
k-means trial 0
iter

iteration
iteration
iteration
iteration
k-means trial 0
iteration
iteration
iteration
iteration
iteration
iteration
iteration
iteration
k-means trial 0
iteration
iteration
iteration
iteration
k-means trial 0
iteration
iteration
iteration
iteration
iteration
iteration
iteration
iteration
iteration
k-means trial 0
iteration
iteration
iteration
iteration
iteration
k-means trial 0
iteration
iteration
iteration
iteration
iteration
iteration
iteration
iteration
k-means trial 0
iteration
iteration
iteration
iteration
iteration
iteration
iteration
k-means trial 0
iteration
iteration
iteration
iteration
iteration
iteration
iteration
iteration
k-means trial 0
iteration
iteration
iteration
iteration
iteration
k-means trial 0
iteration
iteration
iteration
iteration
iteration
iteration
iteration
iteration
iteration
iteration
iteration
k-means trial 0
iteration
iteration
iteration
iteration
iteration
iteration
k-means trial 0
iteration
iteration
iteration
iteration
iteration
iteration
iteration
iter

iteration
iteration
iteration
iteration
iteration
k-means trial 0
iteration
iteration
iteration
iteration
iteration
k-means trial 0
iteration
iteration
iteration
iteration
iteration
iteration
k-means trial 0
iteration
iteration
iteration
iteration
iteration
k-means trial 0
iteration
iteration
iteration
iteration
iteration
k-means trial 0
iteration
iteration
iteration
iteration
iteration
k-means trial 0
iteration
iteration
iteration
iteration
iteration
iteration
iteration
iteration
iteration
iteration
iteration
iteration
k-means trial 0
iteration
iteration
iteration
iteration
iteration
iteration
iteration
iteration
iteration
iteration
iteration
k-means trial 0
iteration
iteration
iteration
k-means trial 0
iteration
iteration
iteration
iteration
iteration
iteration
iteration
iteration
k-means trial 0
iteration
iteration
iteration
iteration
iteration
iteration
iteration
iteration
iteration
iteration
k-means trial 0
iteration
iteration
iteration
iteration
iteration
k-means trial 0
iteratio

iteration
iteration
iteration
iteration
k-means trial 0
iteration
iteration
iteration
iteration
iteration
iteration
k-means trial 0
iteration
iteration
iteration
iteration
iteration
iteration
iteration
iteration
iteration
iteration
iteration
iteration
iteration
iteration
k-means trial 0
iteration
iteration
iteration
iteration
iteration
iteration
iteration
iteration
iteration
iteration
k-means trial 0
iteration
iteration
iteration
iteration
iteration
iteration
iteration
iteration
iteration
k-means trial 0
iteration
iteration
iteration
iteration
k-means trial 0
iteration
iteration
iteration
iteration
iteration
iteration
iteration
iteration
iteration
iteration
iteration
iteration
iteration
k-means trial 0
iteration
iteration
iteration
iteration
iteration
iteration
iteration
iteration
iteration
iteration
iteration
k-means trial 0
iteration
iteration
iteration
iteration
iteration
k-means trial 0
iteration
iteration
iteration
iteration
iteration
iteration
iteration
iteration
k-means trial 0


k-means trial 0
iteration
iteration
iteration
iteration
iteration
iteration
iteration
iteration
k-means trial 0
iteration
iteration
iteration
iteration
k-means trial 0
iteration
iteration
iteration
iteration
iteration
iteration
iteration
iteration
iteration
k-means trial 0
iteration
iteration
iteration
iteration
iteration
iteration
k-means trial 0
iteration
iteration
iteration
iteration
iteration
iteration
k-means trial 0
iteration
iteration
iteration
iteration
iteration
k-means trial 0
iteration
iteration
iteration
iteration
iteration
k-means trial 0
iteration
iteration
iteration
iteration
iteration
iteration
k-means trial 0
iteration
iteration
iteration
iteration
iteration
iteration
k-means trial 0
iteration
iteration
iteration
iteration
iteration
iteration
iteration
k-means trial 0
iteration
iteration
iteration
iteration
iteration
iteration
iteration
k-means trial 0
iteration
iteration
iteration
iteration
iteration
k-means trial 0
iteration
iteration
iteration
iteration
iteration
it

iteration
iteration
iteration
iteration
iteration
k-means trial 0
iteration
iteration
iteration
iteration
iteration
k-means trial 0
iteration
iteration
iteration
iteration
iteration
iteration
iteration
iteration
iteration
iteration
k-means trial 0
iteration
iteration
iteration
iteration
iteration
iteration
iteration
iteration
k-means trial 0
iteration
iteration
iteration
iteration
iteration
k-means trial 0
iteration
iteration
iteration
k-means trial 0
iteration
iteration
iteration
iteration
iteration
k-means trial 0
iteration
iteration
iteration
k-means trial 0
iteration
iteration
iteration
iteration
iteration
iteration
k-means trial 0
iteration
iteration
iteration
iteration
k-means trial 0
iteration
iteration
iteration
iteration
iteration
iteration
iteration
k-means trial 0
iteration
iteration
iteration
iteration
k-means trial 0
iteration
iteration
iteration
k-means trial 0
iteration
iteration
iteration
iteration
iteration
iteration
k-means trial 0
iteration
iteration
iteration
iterat

iteration
k-means trial 0
iteration
iteration
iteration
iteration
iteration
iteration
k-means trial 0
iteration
iteration
iteration
iteration
iteration
iteration
iteration
k-means trial 0
iteration
iteration
iteration
iteration
iteration
iteration
iteration
k-means trial 0
iteration
iteration
iteration
iteration
iteration
iteration
iteration
iteration
iteration
iteration
k-means trial 0
iteration
iteration
iteration
iteration
iteration
k-means trial 0
iteration
iteration
iteration
iteration
iteration
iteration
iteration
iteration
iteration
iteration
k-means trial 0
iteration
iteration
iteration
iteration
iteration
iteration
iteration
iteration
iteration
iteration
iteration
iteration
k-means trial 0
iteration
iteration
iteration
iteration
iteration
iteration
iteration
k-means trial 0
iteration
iteration
iteration
iteration
iteration
iteration
k-means trial 0
iteration
iteration
iteration
iteration
iteration
k-means trial 0
iteration
iteration
iteration
iteration
iteration
k-means trial 

iteration
iteration
iteration
iteration
iteration
k-means trial 0
iteration
iteration
iteration
iteration
iteration
iteration
k-means trial 0
iteration
iteration
iteration
iteration
iteration
iteration
k-means trial 0
iteration
iteration
iteration
iteration
iteration
iteration
iteration
k-means trial 0
iteration
iteration
iteration
iteration
iteration
iteration
iteration
k-means trial 0
iteration
iteration
iteration
iteration
iteration
iteration
iteration
k-means trial 0
iteration
iteration
iteration
iteration
iteration
iteration
iteration
k-means trial 0
iteration
iteration
iteration
iteration
iteration
iteration
k-means trial 0
iteration
iteration
iteration
k-means trial 0
iteration
iteration
iteration
iteration
iteration
iteration
k-means trial 0
iteration
iteration
iteration
iteration
iteration
iteration
iteration
iteration
k-means trial 0
iteration
iteration
iteration
iteration
k-means trial 0
iteration
iteration
iteration
iteration
iteration
iteration
iteration
iteration
k-means 

iteration
iteration
iteration
k-means trial 0
iteration
iteration
iteration
k-means trial 0
iteration
iteration
iteration
iteration
iteration
iteration
iteration
iteration
k-means trial 0
iteration
iteration
iteration
iteration
iteration
k-means trial 0
iteration
iteration
iteration
iteration
iteration
iteration
iteration
iteration
iteration
iteration
iteration
iteration
iteration
k-means trial 0
iteration
iteration
iteration
iteration
iteration
iteration
iteration
iteration
iteration
iteration
iteration
iteration
iteration
iteration
iteration
k-means trial 0
iteration
iteration
iteration
iteration
iteration
iteration
iteration
iteration
iteration
iteration
iteration
iteration
iteration
iteration
iteration
iteration
iteration
iteration
iteration
k-means trial 0
iteration
iteration
iteration
iteration
iteration
iteration
iteration
iteration
k-means trial 0
iteration
iteration
iteration
iteration
iteration
iteration
iteration
k-means trial 0
iteration
iteration
iteration
iteration
iterat

In [9]:
print(x)

0.576271186440678


In [10]:
abc2=random_shff(actionconfidential,actionembezzlement,actiondataleak,actionpasswords)
x2=k_nearest_neightbor_accuracy(abc2[0],abc2[1],abc2[2],12,cut_number=50)
print(x2)

Validating: The accuracy when K= 1 is 0.608098833218943
Validating: The accuracy when K= 2 is 0.6129032258064516
Validating: The accuracy when K= 3 is 0.6177076183939602
Validating: The accuracy when K= 4 is 0.6280027453671929
Validating: The accuracy when K= 5 is 0.6273164035689773
Validating: The accuracy when K= 6 is 0.6280027453671929
Validating: The accuracy when K= 7 is 0.6280027453671929
Validating: The accuracy when K= 8 is 0.6280027453671929
Validating: The accuracy when K= 9 is 0.6280027453671929
Validating: The accuracy when K= 10 is 0.628231525966598
Validating: The highest accuracy is 0.628231525966598 The corresponding k is 10
0.5430379746835443


In [11]:
abc3=random_shff(actionconfidential,actionembezzlement,actiondataleak,actionpasswords)
x3=Weighted_average_accuracy(abc3[0],abc3[1],abc3[2],12,cut_number=50)
print(x3)

Validating: The accuracy when K= 1 is 0.6181683899556869
Validating: The accuracy when K= 2 is 0.6181683899556869
Validating: The accuracy when K= 3 is 0.6214918759231906
Validating: The accuracy when K= 4 is 0.6307237813884786
Validating: The accuracy when K= 5 is 0.6292466765140325
Validating: The accuracy when K= 6 is 0.628138847858198
Validating: The accuracy when K= 7 is 0.6285081240768094
Validating: The accuracy when K= 8 is 0.6285081240768094
Validating: The accuracy when K= 9 is 0.6283604135893649
Validating: The accuracy when K= 10 is 0.6282619399310685
Validating: The highest accuracy is 0.6307237813884786 The corresponding k is 4
0.6258673026886383


## HR@k k=2

In [12]:
def k_mean_HR(train,vali,test,num_actions,k,k2=4,cut_number=50):
    # train,vali and test are trainning set, validation set and test set. Cut_number is the starting point for testing sequence. 
    # if starting from the beginning, change cut_number to 0
    # K2=4 because for UI data there are four tasks
    # k is HR@k's k
    success_num=0
    total=0
    for i in vali:
        test.append(i)
    Pairwise=create_user_transit_matrix(test,num_actions)
    acuracy=[]
    returnk=[]
    highest=0
    index=-1
    
    
    
    for i in train:
        length=len(i)
        for j in range(cut_number,length):
            total+=1
            if i[j] in pairwise_k_mean(i[:j], test, num_actions,k2,Pairwise)[-k:] :
                success_num+=1
    return success_num/total
def k_nearest_HR(train,vali,test,num_actions,k2,cut_number=50):
     # train,vali and test are trainning set, validation set and test set. Cut_number is the starting point for testing sequence. 
    # if starting from the beginning, change cut_number to 0
    # K2=4 is the number of neighbors

    success_num=0
    total=0
    Pairwise=create_user_transit_matrix(test,num_actions)
    acuracy=[]
    returnk=[]
    highest=0
    index=-1
    for k in range(1,11):
        for i in vali:
            length=len(i)
            for j in range(cut_number,length):
                total+=1
                if i[j] in pairwise_k_nearst_neighbor_cosine(k,i[:j], test, num_actions,Pairwise)[-k2:]:
                    
                    success_num+=1
        print('Validating: The accuracy when K=',k,'is', success_num/total)
        if success_num/total>highest:
            highest=success_num/total
            index=k
            success_num=0
            total=0
    print("Validating: The highest accuracy is",highest,'The corresponding k is',index)
    for i in vali:
        test.append(i)
    for i in train:
        length=len(i)
        for j in range(cut_number,length):
            total+=1
            if i[j] in pairwise_k_nearst_neighbor_cosine(k,i[:j], test, num_actions,Pairwise)[-k2:]:
                success_num+=1
    return success_num/total
def weighted_HR(train,vali,test,num_actions,k2,cut_number=50):
    # train,vali and test are trainning set, validation set and test set. Cut_number is the starting point for testing sequence. 
    # if starting from the beginning, change cut_number to 0
    # K2=4 is the number of neighbors
    success_num=0
    total=0
    Pairwise=create_user_transit_matrix(test,num_actions)
    acuracy=[]
    returnk=[]
    highest=0
    index=-1
    for k in range(1,11):
        for i in vali:
            length=len(i)
            for j in range(cut_number,length-1):
                total+=1
                if i[j] in pairwise_weighted_average(k,i[:j], test, num_actions,Pairwise)[-k2:]:
                    
                    success_num+=1
        print('Validating: The accuracy when K=',k,'is', success_num/total)
        if success_num/total>highest:
            highest=success_num/total
            index=k
            success_num=0
            total=0
    print("Validating: The highest accuracy is",highest,'The corresponding k is',index)
    for i in vali:
        test.append(i)
    for i in train:
        length=len(i)
        for j in range(cut_number,length-1):
            total+=1
            if i[j] in pairwise_weighted_average(k,i[:j], test, num_actions,Pairwise)[-k2:]:
                success_num+=1
    return success_num/total

In [13]:
abc4=random_shff(actionconfidential,actionembezzlement,actiondataleak,actionpasswords)
x4=k_mean_HR(abc4[0],abc4[1],abc4[2],12,2,k2=4,cut_number=50)




k-means trial 0
iteration
iteration
iteration
iteration
iteration
iteration
iteration
iteration
iteration
iteration
k-means trial 0
iteration
iteration
iteration
iteration
iteration
iteration
k-means trial 0
iteration
iteration
iteration
iteration
iteration
iteration
iteration
iteration
iteration
iteration
iteration
iteration
k-means trial 0
iteration
iteration
iteration
iteration
iteration
iteration
k-means trial 0
iteration
iteration
iteration
iteration
iteration
iteration
iteration
iteration
k-means trial 0
iteration
iteration
iteration
iteration
k-means trial 0
iteration
iteration
iteration
iteration
iteration
iteration
iteration
k-means trial 0
iteration
iteration
iteration
iteration
iteration
iteration
iteration
iteration
iteration
k-means trial 0
iteration
iteration
iteration
iteration
iteration
k-means trial 0
iteration
iteration
iteration
iteration
iteration
iteration
iteration
k-means trial 0
iteration
iteration
iteration
iteration
iteration
iteration
iteration
k-means trial 

iteration
iteration
k-means trial 0
iteration
iteration
iteration
iteration
iteration
k-means trial 0
iteration
iteration
iteration
iteration
iteration
iteration
iteration
iteration
iteration
iteration
iteration
k-means trial 0
iteration
iteration
iteration
iteration
iteration
iteration
iteration
k-means trial 0
iteration
iteration
iteration
iteration
iteration
iteration
iteration
iteration
iteration
iteration
iteration
iteration
iteration
iteration
iteration
iteration
k-means trial 0
iteration
iteration
iteration
iteration
k-means trial 0
iteration
iteration
iteration
iteration
iteration
iteration
k-means trial 0
iteration
iteration
iteration
iteration
iteration
iteration
k-means trial 0
iteration
iteration
iteration
iteration
iteration
iteration
iteration
iteration
k-means trial 0
iteration
iteration
iteration
iteration
iteration
iteration
k-means trial 0
iteration
iteration
iteration
iteration
iteration
iteration
iteration
iteration
iteration
iteration
k-means trial 0
iteration
iter

iteration
iteration
iteration
iteration
iteration
iteration
iteration
iteration
iteration
iteration
k-means trial 0
iteration
iteration
iteration
iteration
iteration
iteration
iteration
iteration
iteration
iteration
iteration
iteration
iteration
iteration
k-means trial 0
iteration
iteration
iteration
iteration
iteration
iteration
k-means trial 0
iteration
iteration
iteration
iteration
iteration
iteration
iteration
iteration
iteration
iteration
k-means trial 0
iteration
iteration
iteration
iteration
iteration
k-means trial 0
iteration
iteration
iteration
iteration
iteration
iteration
iteration
iteration
k-means trial 0
iteration
iteration
iteration
iteration
iteration
iteration
iteration
iteration
iteration
k-means trial 0
iteration
iteration
iteration
iteration
iteration
iteration
iteration
iteration
iteration
iteration
k-means trial 0
iteration
iteration
iteration
iteration
iteration
iteration
iteration
iteration
iteration
iteration
k-means trial 0
iteration
iteration
iteration
iterat

iteration
iteration
iteration
k-means trial 0
iteration
iteration
iteration
iteration
iteration
iteration
iteration
iteration
iteration
iteration
k-means trial 0
iteration
iteration
iteration
iteration
iteration
iteration
iteration
k-means trial 0
iteration
iteration
iteration
iteration
iteration
k-means trial 0
iteration
iteration
iteration
iteration
iteration
iteration
k-means trial 0
iteration
iteration
iteration
iteration
iteration
k-means trial 0
iteration
iteration
iteration
iteration
iteration
k-means trial 0
iteration
iteration
iteration
iteration
iteration
iteration
k-means trial 0
iteration
iteration
iteration
iteration
iteration
iteration
iteration
iteration
iteration
iteration
iteration
iteration
k-means trial 0
iteration
iteration
iteration
iteration
iteration
k-means trial 0
iteration
iteration
iteration
iteration
iteration
iteration
k-means trial 0
iteration
iteration
iteration
iteration
k-means trial 0
iteration
iteration
iteration
iteration
iteration
iteration
k-means 

iteration
iteration
iteration
iteration
k-means trial 0
iteration
iteration
iteration
iteration
k-means trial 0
iteration
iteration
iteration
iteration
iteration
iteration
iteration
iteration
iteration
iteration
k-means trial 0
iteration
iteration
iteration
iteration
iteration
iteration
iteration
k-means trial 0
iteration
iteration
iteration
iteration
iteration
k-means trial 0
iteration
iteration
iteration
iteration
iteration
k-means trial 0
iteration
iteration
iteration
iteration
iteration
iteration
iteration
k-means trial 0
iteration
iteration
iteration
iteration
iteration
iteration
iteration
iteration
iteration
k-means trial 0
iteration
iteration
iteration
iteration
iteration
iteration
iteration
iteration
iteration
iteration
iteration
iteration
iteration
iteration
k-means trial 0
iteration
iteration
iteration
iteration
iteration
iteration
iteration
iteration
iteration
iteration
iteration
iteration
k-means trial 0
iteration
iteration
iteration
iteration
iteration
iteration
k-means tr

...

iteration
k-means trial 0
iteration
iteration
iteration
k-means trial 0
iteration
iteration
iteration
k-means trial 0
iteration
iteration
iteration
iteration
iteration
k-means trial 0
iteration
iteration
iteration
iteration
iteration
iteration
k-means trial 0
iteration
iteration
iteration
iteration
iteration
iteration
iteration
iteration
iteration
iteration
iteration
iteration
iteration
iteration
k-means trial 0
iteration
iteration
iteration
k-means trial 0
iteration
iteration
iteration
iteration
iteration
iteration
iteration
iteration
k-means trial 0
iteration
iteration
iteration
iteration
iteration
iteration
iteration
iteration
iteration
iteration
iteration
k-means trial 0
iteration
iteration
iteration
iteration
iteration
k-means trial 0
iteration
iteration
iteration
iteration
iteration
k-means trial 0
iteration
iteration
iteration
iteration
iteration
iteration
iteration
k-means trial 0
iteration
iteration
iteration
iteration
iteration
iteration
iteration
iteration
iteration
iteratio

iteration
iteration
iteration
iteration
iteration
k-means trial 0
iteration
iteration
iteration
iteration
iteration
k-means trial 0
iteration
iteration
iteration
iteration
iteration
k-means trial 0
iteration
iteration
iteration
iteration
iteration
iteration
iteration
iteration
iteration
iteration
iteration
k-means trial 0
iteration
iteration
iteration
iteration
iteration
iteration
iteration
iteration
iteration
iteration
iteration
iteration
k-means trial 0
iteration
iteration
iteration
iteration
iteration
iteration
k-means trial 0
iteration
iteration
iteration
iteration
iteration
k-means trial 0
iteration
iteration
iteration
iteration
iteration
iteration
iteration
iteration
k-means trial 0
iteration
iteration
iteration
iteration
iteration
iteration
iteration
iteration
iteration
iteration
iteration
iteration
k-means trial 0
iteration
iteration
iteration
iteration
iteration
iteration
k-means trial 0
iteration
iteration
iteration
iteration
iteration
iteration
iteration
k-means trial 0
iter

iteration
iteration
iteration
iteration
iteration
k-means trial 0
iteration
iteration
iteration
iteration
iteration
iteration
iteration
k-means trial 0
iteration
iteration
iteration
iteration
iteration
iteration
iteration
iteration
k-means trial 0
iteration
iteration
iteration
iteration
iteration
iteration
iteration
iteration
k-means trial 0
iteration
iteration
iteration
iteration
iteration
k-means trial 0
iteration
iteration
iteration
iteration
iteration
iteration
k-means trial 0
iteration
iteration
iteration
iteration
iteration
iteration
iteration
iteration
k-means trial 0
iteration
iteration
iteration
iteration
iteration
iteration
iteration
iteration
iteration
k-means trial 0
iteration
iteration
iteration
iteration
iteration
iteration
iteration
k-means trial 0
iteration
iteration
iteration
iteration
iteration
iteration
k-means trial 0
iteration
iteration
iteration
iteration
iteration
iteration
k-means trial 0
iteration
iteration
iteration
iteration
iteration
iteration
iteration
k-me

iteration
iteration
iteration
iteration
iteration
k-means trial 0
iteration
iteration
iteration
iteration
iteration
iteration
iteration
iteration
k-means trial 0
iteration
iteration
iteration
iteration
iteration
k-means trial 0
iteration
iteration
iteration
iteration
iteration
iteration
iteration
k-means trial 0
iteration
iteration
iteration
iteration
iteration
iteration
iteration
iteration
iteration
k-means trial 0
iteration
iteration
iteration
iteration
k-means trial 0
iteration
iteration
iteration
iteration
iteration
iteration
iteration
k-means trial 0
iteration
iteration
iteration
iteration
k-means trial 0
iteration
iteration
iteration
iteration
iteration
iteration
k-means trial 0
iteration
iteration
iteration
iteration
iteration
k-means trial 0
iteration
iteration
iteration
iteration
iteration
k-means trial 0
iteration
iteration
iteration
iteration
k-means trial 0
iteration
iteration
iteration
iteration
iteration
iteration
k-means trial 0
iteration
iteration
iteration
iteration
it

iteration
iteration
k-means trial 0
iteration
iteration
iteration
iteration
iteration
iteration
k-means trial 0
iteration
iteration
iteration
iteration
iteration
iteration
k-means trial 0
iteration
iteration
iteration
iteration
iteration
k-means trial 0
iteration
iteration
iteration
iteration
iteration
iteration
iteration
iteration
iteration
iteration
iteration
k-means trial 0
iteration
iteration
iteration
iteration
iteration
iteration
iteration
iteration
k-means trial 0
iteration
iteration
iteration
iteration
iteration
iteration
iteration
iteration
iteration
iteration
k-means trial 0
iteration
iteration
iteration
iteration
iteration
iteration
iteration
k-means trial 0
iteration
iteration
iteration
iteration
iteration
iteration
iteration
iteration
k-means trial 0
iteration
iteration
iteration
iteration
iteration
iteration


In [14]:
print(x4)

0.7790346907993967


In [15]:
abc5=random_shff(actionconfidential,actionembezzlement,actiondataleak,actionpasswords)
x5=k_nearest_HR(abc5[0],abc5[1],abc5[2],12,2,cut_number=50)


Validating: The accuracy when K= 1 is 0.7112264763140818
Validating: The accuracy when K= 2 is 0.7293964957819598
Validating: The accuracy when K= 3 is 0.7261518494484102
Validating: The accuracy when K= 4 is 0.7303698896820247
Validating: The accuracy when K= 5 is 0.736534717715769
Validating: The accuracy when K= 6 is 0.7352368591823492
Validating: The accuracy when K= 7 is 0.7349123945489942
Validating: The accuracy when K= 8 is 0.7356694786934891
Validating: The accuracy when K= 9 is 0.735885788449059
Validating: The accuracy when K= 10 is 0.7362751460090851
Validating: The highest accuracy is 0.736534717715769 The corresponding k is 5


In [22]:
print(x5)

0.7352427523576668


In [16]:
abc6=random_shff(actionconfidential,actionembezzlement,actiondataleak,actionpasswords)
x6=weighted_HR(abc6[0],abc6[1],abc6[2],12,2,cut_number=50)


Validating: The accuracy when K= 1 is 0.7654952076677316
Validating: The accuracy when K= 2 is 0.7801916932907348
Validating: The accuracy when K= 3 is 0.7840255591054313
Validating: The accuracy when K= 4 is 0.7769968051118211
Validating: The accuracy when K= 5 is 0.7779552715654952
Validating: The accuracy when K= 6 is 0.7769968051118211
Validating: The accuracy when K= 7 is 0.7749201277955271
Validating: The accuracy when K= 8 is 0.7735463258785943
Validating: The accuracy when K= 9 is 0.7728434504792332
Validating: The accuracy when K= 10 is 0.7720675490643542
Validating: The highest accuracy is 0.7840255591054313 The corresponding k is 3


In [23]:
print(x6)

0.7716758053461275


In [17]:
## Log_Loss

In [18]:
def normalize(v): #normalize the entire vector
    norm=np.linalg.norm(v, ord=1)
    if norm==0:
        norm=np.finfo(v.dtype).eps
    return v/norm
def log_loss_kmean(train,vali,test,num_actions,cut_number=50,k=4):
    logloss=0
    total=0
    for i in vali:
        test.append(i)
    Pairwise=create_user_transit_matrix(test,num_actions)
    Pairwise2=create_user_transit_matrix(train,num_actions)
    acuracy=[]
    returnk=[]
    highest=0
    index=-1

    
    
    
    for i in range(len(train)):
        length=len(train[i])
        for j in range(cut_number,length):
            total+=1
            y_true=np.zeros(num_actions)
            y_pred=[]
            last_index=train[i][j]
            label_num=pairwise_k_mean(train[i][:j], test, num_actions,k,Pairwise)[-1]
            y_true[label_num]=1
            Pro_dis=normalize(Pairwise2[i][last_index*num_actions:(last_index+1)*num_actions])
            for k in Pro_dis:
                temp=[]
                temp.append(1-k)
                temp.append(k)
                y_pred.append(temp)
            logloss+=log_loss(y_true, y_pred) 
            
            
           
                
    return logloss/total
def log_loss_knearest(train,vali,test,num_actions,cut_number=50):
    logloss=0
    total=0
    
    Pairwise=create_user_transit_matrix(test,num_actions)
    Pairwise2=create_user_transit_matrix(train,num_actions)
    acuracy=[]
    returnk=[]
    highest=0
    index=-1
#     print(Pairwise)
    

    for k in range(1,11):
        for i in range(len(train)):
            length=len(train[i])
            for j in range(cut_number,length-1):
                total+=1
                y_true=np.zeros(num_actions)
                y_pred=[]
                last_index=train[i][j]
                label_num=pairwise_k_nearst_neighbor_cosine(k,train[i][:(j+1)], test, num_actions,Pairwise)[-1]
                y_true[label_num]=1
                Pro_dis=normalize(Pairwise2[i][last_index*num_actions:(last_index+1)*num_actions])
                for p in Pro_dis:
                    temp=[]
                    temp.append(1-p)
                    temp.append(p)
                    y_pred.append(temp)
                logloss+=log_loss(y_true, y_pred) 
        if logloss/total>highest:
            highest=logloss/total
            index=k
        print('Validating: The logloss when K=',k,'is', logloss/total)
        logloss=0
        total=0
    print('Validating: The highest logloss when K=',index,'is', highest)
    for i in train:
        test.append(i)       
    for i in range(len(vali)):
            length=len(vali[i])
            for j in range(cut_number,length-1):
                total+=1
                y_true=np.zeros(num_actions)
                y_pred=[]
                last_index=vali[i][j]
                label_num=pairwise_k_nearst_neighbor_cosine(index,vali[i][:(j+1)], test, num_actions,Pairwise)[-1]
                y_true[label_num]=1
                Pro_dis=normalize(Pairwise2[i][last_index*num_actions:(last_index+1)*num_actions])
                for p in Pro_dis:
                    temp=[]
                    temp.append(1-p)
                    temp.append(p)
                    y_pred.append(temp)
                logloss+=log_loss(y_true, y_pred)       
           
                
    return logloss/total
def log_loss_Weighted_Average(train,vali,test,num_actions,cut_number=50):
    logloss=0
    total=0
    
    Pairwise=create_user_transit_matrix(test,num_actions)
    Pairwise2=create_user_transit_matrix(train,num_actions)
    acuracy=[]
    returnk=[]
    highest=0
    index=-1
#     print(Pairwise)
    

    for k in range(1,11):
        for i in range(len(train)):
            length=len(train[i])
            for j in range(cut_number,length-1):
                total+=1
                y_true=np.zeros(num_actions)
                y_pred=[]
                last_index=train[i][j]
                label_num=pairwise_k_nearst_neighbor_cosine(k,train[i][:(j+1)], test, num_actions,Pairwise)[-1]
                y_true[label_num]=1
                Pro_dis=normalize(Pairwise2[i][last_index*num_actions:(last_index+1)*num_actions])
                for p in Pro_dis:
                    temp=[]
                    temp.append(1-p)
                    temp.append(p)
                    y_pred.append(temp)
                logloss+=log_loss(y_true, y_pred) 
        if logloss/total>highest:
            highest=logloss/total
            index=k
        print('Validating: The logloss when K=',k,'is', logloss/total)
        logloss=0
        total=0
    print('Validating: The highest logloss when K=',index,'is', highest)
    for i in train:
        test.append(i)       
    for i in range(len(vali)):
            length=len(vali[i])
            for j in range(cut_number,length-1):
                total+=1
                y_true=np.zeros(num_actions)
                y_pred=[]
                last_index=vali[i][j]
                label_num=pairwise_weighted_average(index,vali[i][:(j+1)], test, num_actions,Pairwise)[-1]
                y_true[label_num]=1
                Pro_dis=normalize(Pairwise2[i][last_index*num_actions:(last_index+1)*num_actions])
                for p in Pro_dis:
                    temp=[]
                    temp.append(1-p)
                    temp.append(p)
                    y_pred.append(temp)
                logloss+=log_loss(y_true, y_pred)       
           
                
    return logloss/total

In [19]:
abc7=random_shff(actionconfidential,actionembezzlement,actiondataleak,actionpasswords)
x7=log_loss_knearest(abc7[0],abc7[1],abc7[2],12)

Validating: The logloss when K= 1 is 0.213371183815
Validating: The logloss when K= 2 is 0.191960337951
Validating: The logloss when K= 3 is 0.178484730362
Validating: The logloss when K= 4 is 0.176135288345
Validating: The logloss when K= 5 is 0.180043312631
Validating: The logloss when K= 6 is 0.17792876792
Validating: The logloss when K= 7 is 0.17819034802
Validating: The logloss when K= 8 is 0.175005092015
Validating: The logloss when K= 9 is 0.162901862172
Validating: The logloss when K= 10 is 0.162901862172
Validating: The highest logloss when K= 1 is 0.213371183815


ValueError: data type <class 'numpy.int32'> not inexact

In [21]:
abc8=random_shff(actionconfidential,actionembezzlement,actiondataleak,actionpasswords)
x8=log_loss_Weighted_Average(abc8[0],abc8[1],abc8[2],12)


Validating: The logloss when K= 1 is 0.169678572265
Validating: The logloss when K= 2 is 0.160534589906
Validating: The logloss when K= 3 is 0.145306247985
Validating: The logloss when K= 4 is 0.151499725402
Validating: The logloss when K= 5 is 0.144619788227
Validating: The logloss when K= 6 is 0.142162522773
Validating: The logloss when K= 7 is 0.139467742701
Validating: The logloss when K= 8 is 0.139665789068
Validating: The logloss when K= 9 is 0.139777966287
Validating: The logloss when K= 10 is 0.137314744098
Validating: The highest logloss when K= 1 is 0.169678572265


ValueError: data type <class 'numpy.int32'> not inexact